In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import

In [195]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()


from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer


import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import random

import warnings
warnings.filterwarnings(action='ignore') 

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cuda')

## Hyperparameter setting

In [196]:
CFG = {
    'EPOCHS': 66,
    'LEARNING_RATE':0.001,
    'BATCH_SIZE':512,
    'SEED':42
}

## Fixed RandomSeed

In [197]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])

## Data Load

In [198]:
data_path = '/content/drive/MyDrive/Colab Notebooks/data/건설기계/'
train = pd.read_csv(f'{data_path}train.csv')
test = pd.read_csv(f'{data_path}test.csv')

In [199]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14095 entries, 0 to 14094
Data columns (total 54 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   14095 non-null  object 
 1   COMPONENT_ARBITRARY  14095 non-null  object 
 2   ANONYMOUS_1          14095 non-null  int64  
 3   YEAR                 14095 non-null  int64  
 4   SAMPLE_TRANSFER_DAY  14095 non-null  int64  
 5   ANONYMOUS_2          14095 non-null  int64  
 6   AG                   14095 non-null  int64  
 7   AL                   14095 non-null  int64  
 8   B                    14095 non-null  int64  
 9   BA                   14095 non-null  int64  
 10  BE                   14095 non-null  int64  
 11  CA                   14095 non-null  int64  
 12  CD                   12701 non-null  float64
 13  CO                   14095 non-null  int64  
 14  CR                   14095 non-null  int64  
 15  CU                   14095 non-null 

In [200]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6041 entries, 0 to 6040
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   6041 non-null   object 
 1   COMPONENT_ARBITRARY  6041 non-null   object 
 2   ANONYMOUS_1          6041 non-null   int64  
 3   YEAR                 6041 non-null   int64  
 4   ANONYMOUS_2          6041 non-null   int64  
 5   AG                   6041 non-null   int64  
 6   CO                   6041 non-null   int64  
 7   CR                   6041 non-null   int64  
 8   CU                   6041 non-null   int64  
 9   FE                   6041 non-null   int64  
 10  H2O                  6041 non-null   float64
 11  MN                   6041 non-null   int64  
 12  MO                   6041 non-null   int64  
 13  NI                   6041 non-null   int64  
 14  PQINDEX              6041 non-null   int64  
 15  TI                   6041 non-null   i

## Data Preprocessing
#### 1. 결측치 처리
#### 2. Train / Validation 분할
#### 3. Data label-encoding, scaling

In [201]:
categorical_features = ['COMPONENT_ARBITRARY', 'YEAR']
# Inference(실제 진단 환경)에 사용하는 컬럼
test_stage_features = ['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR' , 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN']

In [202]:
#train결측치처리
#train['FH2O'] = train['FH2O'].fillna(train['FH2O'].mean()) -> 0.59158
#train['FNOX'] = train['FNOX'].fillna(train['FNOX'].mean()) -> 0.58492
#train['CD'] = train['CD'].fillna(train['CD'].mean()) ->0.59 제출 시 0.58
#train['FOXID'] = train['FOXID'].fillna(train['FOXID'].mean()) -> 0.58747
#train['FSO4'] = train['FSO4'].fillna(train['FSO4'].mean()) -> 0.59017
# train['FTBN'] = train['FTBN'].fillna(train['FTBN'].mean())
# train['FOPTIMETHGLY'] = train['FOPTIMETHGLY'].fillna(train['FOPTIMETHGLY'].mean())
# train['FUEL'] = train['FUEL'].fillna(train['FUEL'].mean())
#train['K'] = train['K'].fillna(train['K'].mean()) -> 0.58
# train['SOOTPERCENTAGE'] = train['SOOTPERCENTAGE'].fillna(train['SOOTPERCENTAGE'].mean())
# train['U100'] = train['U100'].fillna(train['U100'].mean())
# train['U75'] = train['U75'].fillna(train['U75'].mean())
# train['U50'] = train['U50'].fillna(train['U50'].mean())
# train['U25'] = train['U25'].fillna(train['U25'].mean())
# train['U20'] = train['U20'].fillna(train['U20'].mean())
# train['U14'] = train['U14'].fillna(train['U14'].mean())
# train['U6'] = train['U6'].fillna(train['U6'].mean())
# train['V100'] = train['V100'].fillna(train['V100'].mean())


In [203]:
#train drop 실험
#train.drop(columns = ['FH2O'], inplace = True)
# train.drop(columns = ['FNOX'], inplace = True)
# train.drop(columns = ['CD'], inplace = True)
# train.drop(columns = ['FOXID'], inplace = True)
# train.drop(columns = ['FSO4'], inplace = True)
# train.drop(columns = ['FTBN'], inplace = True)
# train.drop(columns = ['FOPTIMETHGLY'], inplace = True)
# train.drop(columns = ['K'], inplace = True)
# train.drop(columns = ['FUEL'], inplace = True)
# train.drop(columns = ['SOOTPERCENTAGE'], inplace = True)
# train.drop(columns = ['U100'], inplace = True)
# train.drop(columns = ['U75'], inplace = True)
# train.drop(columns = ['U50'], inplace = True)
# train.drop(columns = ['U25'], inplace = True)
# train.drop(columns = ['U20'], inplace = True)
# train.drop(columns = ['U14'], inplace = True)
# train.drop(columns = ['U6'], inplace = True)
# train.drop(columns = ['V100'], inplace = True)

In [204]:
#결측치 0값 대체
train = train.fillna(0)
test = test.fillna(0) #test에 결측치 없음
test_size = 0.15
threshold_num = 0.35

In [205]:
all_X = train.drop(['ID', 'Y_LABEL'], axis = 1)
all_y = train['Y_LABEL']

test = test.drop(['ID'], axis = 1)

train_X, val_X, train_y, val_y = train_test_split(all_X, all_y, test_size=test_size, random_state=CFG['SEED'], stratify=all_y)

In [206]:
test_size

0.15

In [207]:
def get_values(value):
    return value.values.reshape(-1, 1)

for col in train_X.columns:
    if col not in categorical_features:
        scaler = StandardScaler()
        # scaler = RobustScaler()
        # scaler = Normalizer()
        train_X[col] = scaler.fit_transform(get_values(train_X[col]))
        val_X[col] = scaler.transform(get_values(val_X[col]))
        if col in test.columns:
            test[col] = scaler.transform(get_values(test[col]))
            
le = LabelEncoder()
for col in categorical_features:    
    train_X[col] = le.fit_transform(train_X[col])
    val_X[col] = le.transform(val_X[col])
    if col in test.columns:
        test[col] = le.transform(test[col])

## CustomDataset

In [208]:
test

,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,0,-0.221718,9,-0.342047,-0.149112,-0.118407,-0.111024,-0.258083,-0.352127,-0.042697,-0.245312,-0.401161,-0.195243,-0.262633,-0.099021,-0.106732,-0.360490,0.943049
1,2,-0.085313,4,-0.342047,-0.149112,-0.118407,-0.046177,-0.258083,0.224228,-0.042697,0.021153,-0.401161,-0.195243,1.524088,0.040787,-0.106732,0.339656,-1.084054
2,1,-0.269301,3,-0.342047,-0.149112,-0.118407,-0.111024,-0.142604,-0.367294,-0.042697,-0.245312,-0.401161,-0.195243,-0.261977,-0.099021,-0.106732,-1.284842,0.234784
3,2,-0.400268,2,-0.342047,-0.149112,-0.118407,-0.013754,-0.234987,-0.024948,-0.042697,0.109974,-0.351161,-0.195243,4.986600,-0.099021,-0.106732,0.652363,-0.930002
4,1,1.145278,6,-0.342047,-0.149112,-0.118407,-0.111024,-0.219590,-0.349960,-0.042697,-0.245312,-0.401161,-0.195243,-0.258695,-0.099021,-0.106732,-0.909201,-0.225494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,2,-0.330026,7,-0.342047,-0.149112,-0.118407,-0.013754,0.735034,1.890458,-0.042697,5.528076,-0.317828,0.342893,1.042947,-0.099021,-0.106732,-0.755798,1.078315
6037,2,0.217633,9,-0.342047,-0.149112,-0.118407,0.051092,-0.250384,1.216599,-0.042697,0.198795,-0.401161,-0.195243,0.185688,-0.099021,-0.106732,0.554027,-1.082175
6038,2,0.261591,7,-0.342047,-0.149112,-0.118407,-0.111024,-0.265782,-0.263290,-0.042697,-0.245312,-0.401161,-0.195243,-0.223906,-0.099021,-0.106732,3.787290,-1.082175
6039,1,-0.409332,6,-0.342047,-0.149112,-0.118407,-0.111024,0.211530,-0.373794,-0.042697,-0.245312,-0.401161,-0.195243,-0.265915,-0.099021,-0.106732,-1.151106,-0.208586


In [209]:
class CustomDataset(Dataset):
    def __init__(self, data_X, data_y, distillation=False):
        super(CustomDataset, self).__init__()
        self.data_X = data_X
        self.data_y = data_y
        self.distillation = distillation
        
    def __len__(self):
        return len(self.data_X)
    
    def __getitem__(self, index):
        if self.distillation:
            # 지식 증류 학습 시
            teacher_X = torch.Tensor(self.data_X.iloc[index])
            student_X = torch.Tensor(self.data_X[test_stage_features].iloc[index])
            y = self.data_y.values[index]
            return teacher_X, student_X, y
        else:
            if self.data_y is None:
                test_X = torch.Tensor(self.data_X.iloc[index])
                return test_X
            else:
                teacher_X = torch.Tensor(self.data_X.iloc[index])
                y = self.data_y.values[index]
                return teacher_X, y

In [210]:
train_dataset = CustomDataset(train_X, train_y, False)
val_dataset = CustomDataset(val_X, val_y, False)

In [211]:
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False)

## Define Teacher Model

In [212]:
class Teacher(nn.Module):
    def __init__(self):
        super(Teacher, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(in_features=52, out_features=256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
            nn.Linear(in_features=256, out_features=1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(),
            nn.Linear(in_features=1024, out_features=256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
            nn.Linear(in_features=256, out_features=1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        output = self.classifier(x)
        return output

## Teacher Train / Validation

In [213]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)

    best_score = 0
    best_model = None
    criterion = nn.BCELoss().to(device)

    for epoch in range(CFG["EPOCHS"]):
        train_loss = []
  
        model.train()
        for X, y in tqdm(train_loader):
            X = X.float().to(device)
            y = y.float().to(device)
            
            
            optimizer.zero_grad()
            
            y_pred = model(X)
            
            loss = criterion(y_pred, y.reshape(-1, 1))
            loss.backward()
            
            optimizer.step()

            train_loss.append(loss.item())

        val_loss, val_score = validation_teacher(model, val_loader, criterion, device)
        print(f'Epoch [{epoch}], Train Loss : [{np.mean(train_loss) :.5f}] Val Loss : [{np.mean(val_loss) :.5f}] Val F1 Score : [{val_score:.5f}]')

        if scheduler is not None:
            scheduler.step(val_score)
            
        if best_score < val_score:
            best_model = model
            best_score = val_score
        writer.add_scalar("Loss/train", np.mean(train_loss), epoch)
        
    return best_model 

In [214]:
def competition_metric(true, pred):
    return f1_score(true, pred, average="macro")


def validation_teacher(model, val_loader, criterion, device):
    model.eval()

    val_loss = []
    pred_labels = []
    true_labels = []
    threshold = threshold_num
    
    with torch.no_grad():
        for X, y in tqdm(val_loader):
            X = X.float().to(device)
            y = y.float().to(device)
            
            model_pred = model(X.to(device))
            
            loss = criterion(model_pred, y.reshape(-1, 1))
            val_loss.append(loss.item())      
            
            model_pred = model_pred.squeeze(1).to('cpu')  
            pred_labels += model_pred.tolist()
            true_labels += y.tolist()
        
        pred_labels = np.where(np.array(pred_labels) > threshold, 1, 0)
        val_f1 = competition_metric(true_labels, pred_labels)
    return val_loss, val_f1   

## Run (Teacher Model)

In [215]:
model = Teacher()
model.eval()
optimizer = torch.optim.Adam(model.parameters(), lr=CFG['LEARNING_RATE'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=1, threshold_mode='abs',min_lr=1e-8, verbose=True)

teacher_model = train(model, optimizer, train_loader, val_loader, scheduler, device)
writer.flush()

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch [0], Train Loss : [0.35996] Val Loss : [0.23246] Val F1 Score : [0.77622]


  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch [64], Train Loss : [0.09686] Val Loss : [0.17054] Val F1 Score : [0.81532]


  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch [65], Train Loss : [0.09649] Val Loss : [0.17002] Val F1 Score : [0.81542]


In [216]:
writer.close()

## Define Student Model

In [217]:
class Student(nn.Module):
    def __init__(self):
        super(Student, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(in_features=18, out_features=128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
            nn.Linear(in_features=128, out_features=512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(),
            nn.Linear(in_features=512, out_features=128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
            nn.Linear(in_features=128, out_features=1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        output = self.classifier(x)
        return output

## Define Knowledge distillation Loss

In [218]:
def distillation(student_logits, labels, teacher_logits, alpha):
    distillation_loss = nn.BCELoss()(student_logits, teacher_logits)
    student_loss = nn.BCELoss()(student_logits, labels.reshape(-1, 1))
    return alpha * student_loss + (1-alpha) * distillation_loss

In [219]:
def distill_loss(output, target, teacher_output, loss_fn=distillation, opt=optimizer):
    loss_b = loss_fn(output, target, teacher_output, alpha=0.1)

    if opt is not None:
        opt.zero_grad()
        loss_b.backward()
        opt.step()

    return loss_b.item()

## Student Train / Validation

In [220]:
def student_train(s_model, t_model, optimizer, train_loader, val_loader, scheduler, device):
    s_model.to(device)
    t_model.to(device)
    
    best_score = 0
    best_model = None

    for epoch in range(CFG["EPOCHS"]):
        train_loss = []
        s_model.train()
        t_model.eval()
        
        for X_t, X_s, y in tqdm(train_loader):
            X_t = X_t.float().to(device)
            X_s = X_s.float().to(device)
            y = y.float().to(device)
            
            optimizer.zero_grad()

            output = s_model(X_s)
            with torch.no_grad():
                teacher_output = t_model(X_t)
                
            loss_b = distill_loss(output, y, teacher_output, loss_fn=distillation, opt=optimizer)

            train_loss.append(loss_b)

        val_loss, val_score = validation_student(s_model, t_model, val_loader, distill_loss, device)
        print(f'Epoch [{epoch}], Train Loss : [{np.mean(train_loss) :.5f}] Val Loss : [{np.mean(val_loss) :.5f}] Val F1 Score : [{val_score:.5f}]')
        
        if scheduler is not None:
            scheduler.step(val_score)
            
        if best_score < val_score:
            best_model = s_model
            best_score = val_score
        
    return best_model

In [221]:
def validation_student(s_model, t_model, val_loader, criterion, device):
    s_model.eval()
    t_model.eval()

    val_loss = []
    pred_labels = []
    true_labels = []
    threshold = threshold_num
    
    with torch.no_grad():
        for X_t, X_s, y in tqdm(val_loader):
            X_t = X_t.float().to(device)
            X_s = X_s.float().to(device)
            y = y.float().to(device)
            
            model_pred = s_model(X_s)
            teacher_output = t_model(X_t)
            
            loss_b = distill_loss(model_pred, y, teacher_output, loss_fn=distillation, opt=None)
            val_loss.append(loss_b)
            
            model_pred = model_pred.squeeze(1).to('cpu')
            pred_labels += model_pred.tolist()
            true_labels += y.tolist()
        
        pred_labels = np.where(np.array(pred_labels) > threshold, 1, 0)
        val_f1 = competition_metric(true_labels, pred_labels)
    return val_loss, val_f1    

## Run (Student Model)

In [222]:
train_dataset = CustomDataset(train_X, train_y, True)
val_dataset = CustomDataset(val_X, val_y, True)

train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False)

In [223]:
student_model = Student()
student_model.eval()
optimizer = torch.optim.Adam(student_model.parameters(), lr=CFG['LEARNING_RATE'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=1, threshold_mode='abs',min_lr=1e-8, verbose=True)

best_student_model = student_train(student_model, teacher_model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch [0], Train Loss : [0.39723] Val Loss : [0.34358] Val F1 Score : [0.48247]


  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch [64], Train Loss : [0.26437] Val Loss : [0.27224] Val F1 Score : [0.51619]


  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch [65], Train Loss : [0.26498] Val Loss : [0.27208] Val F1 Score : [0.51587]


## Choose Inference Threshold

In [224]:
def choose_threshold(model, val_loader, device):
    model.to(device)
    model.eval()
    
    thresholds = [0.1, 0.15,0.16,0.17,0.18,0.19,0.2,0.21,0.211,0.212,0.213,0.214,0.215,0.216,0.217,0.218,0.219,0.22,0.23,0.24, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
    pred_labels = []
    true_labels = []
    
    best_score = 0
    best_thr = None
    with torch.no_grad():
        for _, x_s, y in tqdm(iter(val_loader)):
            x_s = x_s.float().to(device)
            y = y.float().to(device)
            
            model_pred = model(x_s)
            
            model_pred = model_pred.squeeze(1).to('cpu')
            pred_labels += model_pred.tolist()
            true_labels += y.tolist()
        
        for threshold in thresholds:
            pred_labels_thr = np.where(np.array(pred_labels) > threshold, 1, 0)
            score_thr = competition_metric(true_labels, pred_labels_thr)
            if best_score < score_thr:
                best_score = score_thr
                best_thr = threshold
    return best_thr, best_score

In [225]:
best_threshold, best_score = choose_threshold(best_student_model, val_loader, device)
print(f'Best Threshold : [{best_threshold}], Score : [{best_score:.5f}]')

  0%|          | 0/5 [00:00<?, ?it/s]

Best Threshold : [0.218], Score : [0.59017]


## Inference

In [226]:
test_datasets = CustomDataset(test, None, False)
test_loaders = DataLoader(test_datasets, batch_size = CFG['BATCH_SIZE'], shuffle=False)

In [227]:
def inference(model, test_loader, threshold, device):
    model.to(device)
    model.eval()
    
    test_predict = []
    with torch.no_grad():
        for x in tqdm(test_loader):
            x = x.float().to(device)
            model_pred = model(x)

            model_pred = model_pred.squeeze(1).to('cpu')
            test_predict += model_pred
        
    test_predict = np.where(np.array(test_predict) > threshold, 1, 0)
    print('Done.')
    return test_predict

In [228]:
preds = inference(best_student_model, test_loaders, best_threshold, device)

  0%|          | 0/12 [00:00<?, ?it/s]

Done.


## Submit

In [229]:
submit = pd.read_csv(f'{data_path}sample_submission.csv')
submit['Y_LABEL'] = preds
submit.head()

,ID,Y_LABEL
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0


In [230]:
ep_num = CFG['EPOCHS'] 
lr_num = CFG['LEARNING_RATE']
bt_num = CFG['BATCH_SIZE']
seed_num = CFG['SEED'] 

In [231]:
submit['Y_LABEL'].mean()

0.06306902830657175

In [232]:
means = submit['Y_LABEL'].mean()

In [233]:
submit.to_csv(data_path+f'{best_score}_{means}_ep{ep_num}_lr{lr_num}_bt{bt_num}_seed{seed_num}_ts{test_size}_th{threshold_num}submit.csv', index=False)

In [ ]:
best_threshold

0.16

In [ ]:
best_score

0.5931021573307127

In [ ]:
!pwd